In [82]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import pickle
import pandas as pd
import numpy as np
from psm_causal_effects import psm_causal_effects

# read data
with open('data.dat') as f:
    data = pickle.load(f)
f.close()

# removing nan rows
for i in range(len(data)):
    data[i] = data[i].dropna()
    data[i] = data[i].reset_index(drop=True)

# removing empty subjects
ind_empty = []
for i in range(len(data)):
    if data[i].shape[0]==0:
        ind_empty.append(i)
data = [i for j, i in enumerate(data) if j not in ind_empty]
print str(len(ind_empty))+' subjects removed due to lack of data'

for i in range(len(data)):
    data[i]['subject'] = pd.Series(i+np.zeros(data[i].shape[0]), index=data[i].index, dtype=int)

# concatenatig into a single dataframe
data = pd.concat(data, axis=0)
data = data.reset_index(drop=True)

# remove extra columns
# data = data.loc[:,['subject','mood','quality']]
data['mood'] = data['mood'].astype(float)
data['mood_prev'] = data['mood_prev'].astype(float)
data['quality'] = data['quality'].astype(float)

md = smf.mixedlm('mood ~ quality', data, groups=data['subject'])
mdf = md.fit() 
print(mdf.summary())


25 subjects removed due to lack of data
          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: mood      
No. Observations: 5984    Method:             REML      
No. Groups:       183     Scale:              0.7463    
Min. group size:  2       Likelihood:         -7941.4713
Max. group size:  125     Converged:          Yes       
Mean group size:  32.7                                  
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      3.895    0.080 48.747 0.000  3.738  4.051
quality        0.261    0.009 29.808 0.000  0.243  0.278
Intercept RE   0.855    0.110                           



In [83]:
md = smf.mixedlm('quality ~ mood_prev', data, groups=data['subject'])
mdf = md.fit() 
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: quality    
No. Observations: 5984    Method:             REML       
No. Groups:       183     Scale:              1.6411     
Min. group size:  2       Likelihood:         -10249.2930
Max. group size:  125     Converged:          Yes        
Mean group size:  32.7                                   
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       3.157    0.121 26.097 0.000  2.920  3.395
mood_prev       0.262    0.018 14.454 0.000  0.226  0.297
Intercept RE    1.073    0.098                           



In [60]:
data = sm.datasets.get_rdataset('dietox', 'geepack').data
md = smf.mixedlm('Weight ~ Time', data, groups=data['Pig'])
mdf = md.fit() 
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Weight    
No. Observations: 861     Method:             REML      
No. Groups:       72      Scale:              11.3668   
Min. group size:  11      Likelihood:         -2404.7753
Max. group size:  12      Converged:          Yes       
Mean group size:  12.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    15.724    0.788  19.952 0.000 14.180 17.269
Time          6.942    0.033 207.939 0.000  6.877  7.008
Intercept RE 40.399    2.166                            

